In [1]:
from utils_project import *

from scipy.stats import chi2_contingency
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

def get_item(item):
    return questionnaire_items[questionnaire_items['item']==item]

# demo_controls = ['demo_age', 'demo_gender_1W', 'demo_race_white', 'iq_score', 'disorder'] 
pd.set_option('display.precision', 3)

# reverse the sign of power_tendency_mean in data
data['power_mean_mean'] = data['power_mean_mean'] * -1
sample_dict = update_sample_dict(data)
# data[['sub_id', 'sample', 'affil_mean_mean', 'power_mean_mean'] + demo_controls + [f'factor_{f}_quartimax_thresh25' for f in ['social', 'mood', 'compulsive']]].to_excel('data.xlsx')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initial n=579
Validation n=255
Combined n=834
Questionnaire items loaded


# OLD &/OR EXPERIMENTAL

### Trying to infer diff. poss. viewpoints

### Distance-social factor relationship from all viewpoints
- POV we selected is close to best
- but also maybe suggests we should assume a participant-by-particiant viewpoint: but how to estimate?

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import minimize
def origin_corr_dist(ori, y, df):
    # to minimize the correlation distance to find the best origin
    a, p = ori
    vecs = []
    for char in character_roles[:5]:
        ap = np.concatenate((df[f'affil_coord_{char}'].values.reshape(-1,1), 
                             df[f'power_coord_{char}'].values.reshape(-1,1)), 1)
        ap_ref = ap - (a, p)
        vecs.append([np.linalg.norm(ap) for ap in ap_ref])

    # calculate correlation distance
    return 1 - scipy.stats.pearsonr(np.mean(vecs, axis=0), df[y])[0]

In [ ]:
# define the space
units = 13 # make odd (13 is regular)
space_corrs = np.zeros((units, units))
adj = int(units/2 - .5) # readjust back to plot
lb, ub = -adj, adj + 1

for s, (sample, df) in enumerate(sample_dict.items()):
    
    # loop over diff. poss. ref. point
    vec_refs = pd.DataFrame()
    for a_adj, p_adj in itertools.product(np.arange(lb, ub), np.arange(lb, ub)):

        # subtract the reference point from each characters coordinates
        vecs = []
        for role in character_roles[:5]:
            ap = np.concatenate((df[f'affil_coord_{role}'].values.reshape(-1,1), 
                                 df[f'power_coord_{role}'].values.reshape(-1,1)),1)
            ap_ref = ap - (a_adj, p_adj) 
            vecs.append([np.linalg.norm(ap_r) for ap_r in ap_ref])

        # calculate correlation
        mask = np.isfinite(df[social_factor])
        mean_vecs = np.mean(vecs, axis=0)
        vec_refs[f'ref_{a_adj}{p_adj}'] = mean_vecs
        space_corrs[adj - p_adj, adj + a_adj] = pearsonr(mean_vecs[mask], df[social_factor][mask])[0] # to plot in same space

    #-----------------------------------------------------------------------------------
    # plot
    #-----------------------------------------------------------------------------------
    
    # space.astype(str) # 0 out uninteresting cells to control annotation (https://stackoverflow.com/questions/60611055/add-annotation-to-specific-cells-in-heatmap)
    fig, ax = plt.subplots(figsize=(4, 4))
    im = plt.imshow(space_corrs, cmap='RdBu_r', zorder=0)
    ax.set_xticks(np.arange(0, units, 1))
    ax.set_yticks(np.arange(0, units, 1))
    ax.set_yticklabels(reversed(np.arange(lb, ub)))
    ax.set_xticklabels(np.arange(lb, ub))
    ax.set_xlabel('Affiliation', fontsize=20)
    ax.set_ylabel('Power', fontsize=20)
    
    # add a colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')    
    cbar.ax.tick_params(labelsize=8) 
    
    plt.tight_layout()
    plt.show()
    
    #     # add first eigenvector
    #     eig_vals, eig_vecs = np.linalg.eigh(space_corrs) # for symmetric matrices
    #     sort = np.argsort(eig_vals)[::-1]
    #     eig_vals = eig_vals[sort]
    #     eig_vecs = eig_vecs[:, sort]
    #     origin = [0,0]
    #     ax.quiver(*origin, *eig_vecs[0], color=['r'], scale=eig_vals[0], zorder=1)
    #     plt.show()

    #-----------------------------------------------------------------------------------
    # best reference frame coordinates
    #-----------------------------------------------------------------------------------

    # # - the point at which if distance is measured from, it maximizes the correlation 
    # best = np.where(space_corrs==np.max(space_corrs)) # best positive correlation
    # best_r = -(best[0][0] - adj) # will be power (y)
    # best_c = best[1][0] - adj # will be affil (x)
    # print(best_c, best_r)

    #-----------------------------------------------------------------------------------
    # which x,y minimize the correlation distance? (maximize positive correlation)
    #-----------------------------------------------------------------------------------
        
    best_ref = minimize(origin_corr_dist, (0, 0), bounds=([-6,6], [-6,6]), 
                        args=(social_factor, df[mask]), method='L-BFGS-B').x
    print(f'{sample} best ref location = {best_ref}')

In [ ]:
for factor in factors:
    mask = np.isfinite(df[factor])
    res = minimize(origin_corr_dist, (0, 0), 
                    bounds=([-6,6], [-6,6]), 
                    args=(factor, df[mask]), 
                    method='L-BFGS-B')
    print(f'{factor}: {np.round(res.x, 2)}, r={np.round(1 - res.fun, 3)}')

In [ ]:
# orthogonality of affil and power across diff levels of social factor
factor_splits = ['lowest', 'low', 'medium', 'high', 'highest']
for f_split in factor_splits:
    split_data = data[data[f'{social_factor}_split'] == f_split]
    r,p = pearsonr(split_data['affil_mean_mean_z'], split_data['power_mean_mean_z'])
    print(f'{f_split}: r={r:.3f}, p={p:.3f}')

# Individual decisions: the specifics of the trajectories...

In [ ]:
from scipy.stats import rankdata
from circ_stats import binary_distances

def pairwise_annak_distances(ranks):
    # https://naturalistic-data.org/content/Intersubject_RSA.html
    # anna karenina distance
    # metric is mean of ranks, normalized by highest possible rank (n_subs)
    n = len(ranks)
    annak = np.zeros((n, n))
    for i, j in itertools.product(range(n), range(n)):
        if i < j:
            annak[i,j] = annak[j,i] = np.mean([ranks[i], ranks[j]])/ n
        elif i==j:
            annak[i,j] = 1
    return annak

def compute_pairwise(X, pairwise_f): 
    n = X.shape[1]
    out = np.zeros((n, n))
    for i in range(0, n):
        x1 = X[i,:][:,np.newaxis]
        for j in range(i, n):
            out[i,j] = out[j, i] = pairwise_f(x1, X[j,:][:,np.newaxis])
    return out


In [ ]:
decision_data = data[[c for c in data.columns if 'decision' in c]].values
decision_rdm = pairwise_distances(decision_data, metric='hamming')

dist_ranks = rankdata(data['pov_2d_dist_mean_mean'].values)
sf_ranks   = rankdata(data[social_factor].values)
decision_rdm_sorted = sort_symm_mat(decision_rdm, dist_ranks)

plot = plot_rdm(sort_symm_mat(decision_rdm, dist_ranks), size=8)
plot = plot_rdm(sort_symm_mat(decision_rdm, sf_ranks), size=8)

## Dots: can we use neutral character's offset from (0,0) to infer social avoidance?
- if yes: suggests high SA ppl might represent others as distant even when there were no affiliation and power interactions
- if no: suggests affiliation and power interactions are required

neither should be considered strong evidence... 

Doesnt seem like it affects much here... suggesting (I guess) that decisions are important

In [ ]:
cols = ['beh_dots_affil_diff_neutral', 'beh_dots_power_diff_neutral', 'dots_pov_dist_neutral']
fig, axs = plt.subplots(1, len(cols), figsize=(len(cols)*4, 4), sharey=True)
for s, sample in enumerate(['Initial', 'Replication']):
    df = sample_dict[sample]
    for c, col in enumerate(cols):
        ax = axs[c]
        sns.regplot(x=df[col], y=df[social_factor], color=sample_colors[s], ax=ax)
        # ols_obj = run_ols([col], social_factor, df, covariates=['demo_age', 'demo_sex_1F', 'demo_race_white', 'iq_score'])[1]
        # p = ols_obj.pvalues[col] / 2
        # print(f'{sample} p={p:3f}')

# Validating the use of factor analysis: regressions w/ individual questionnaires

### OLS
- each questionnaire individually
- questionnaires together
    - also: select questionnaires on first dataset, then replicate relationships

In [ ]:
def get_item_labels(ques_prefix):
    return [c for c in data.columns if (c.split('_')[0] == ques_prefix) & ('score' not in c) & ('_att' not in c)]

# put togehter int a dictionary
behs = ['affil_mean_mean_z', 'power_mean_mean_z', 'pov_2d_dist_mean_mean_z']
ques = ['oci_score', 'sds_score', 'aes_score', 'sss_score', 'lsas_av_score', 'apdis_score', 'zbpd_score', 'bapq_score']
for q in ques: data[f'{q}_z'] = zscore_masked(data[q].values) # zscore the questionnaire scores
sample_dict = reset_sample_dict(data)
ques_scores = [f'{q}_z' for q in ques]

ques_prefix = [q.split('_')[0] for q in ques]
ques_dict   = {'scores': ques_scores, 'items': flatten_lists([get_item_labels(qp) for qp in ques_prefix])}

# run ols
ols_df = pd.DataFrame(columns=['sample', 'predictor_type', 'self-report', 'behavior', 'beta', 'p'])
for sample in ['Initial', 'Replication', 'Combined']:
    df = sample_dict[sample]
    for b in behs:  
        for qtype, qs in ques_dict.items():
            
            # behavior ~ single score/item
            for q in qs:
                ols = run_ols([q], b, df, covariates=demo_controls)[1]
                ols_df.loc[len(ols_df), :] = [sample, f'independent_{qtype}', q, b, ols.params[q], ols.pvalues[q]] 

            # behavior ~ all scores/items
            ols = run_ols(qs, b, df, covariates=demo_controls)[1]
            for q in qs:
                ols_df.loc[len(ols_df), :] = [sample, f'combined_{qtype}', q, b, ols.params[q], ols.pvalues[q]]

In [ ]:
# many items are reverse scored, so these should all be: higher value means more symptom
item_df = ols_df[(ols_df['sample'] == 'Initial') 
                 & (ols_df['predictor_type'] == 'combined_items') 
                 & (ols_df['behavior'] == 'pov_2d_dist_mean_mean_z')]
item_df.sort_values(by=['sample','p'], inplace=True)
item_df.reset_index(drop=True, inplace=True)

# find top items
item_df_ = item_df.iloc[:10]
for qp in ques_prefix: 
    items = item_df_[item_df_['self-report'].str.contains(qp)]['self-report']
    print(f'{qp}: {items.values}')

# find the text
for i, item in enumerate(item_df['self-report']):
    try: 
        item_text = questionnaire_items[questionnaire_items['item'] == item]['text'].values[0]
        item_df.loc[i, 'item_text'] = item_text
    except: 
        item_df.loc[i, 'item_text'] = 'missing'
item_df.head(20)

In [ ]:
def plot_weights(weights, colors, title, ax):
    weights = weights.astype(float)
    w_max = np.round(np.max(np.abs(weights)), 1)
    ax.bar(np.arange(len(weights)), weights, color=colors, edgecolor='black', linewidth=0.5)
    ax.set_title(title, fontsize=10)
    #ax.set_ylim(-w_max-(0.005*w_max), w_max+(0.005*w_max)) 

behs = ['pov_2d_dist_mean_mean_z']

# for plotting
colors = ['red', 'blue', 'purple', 'green', 'lavender', 'grey', 'fuchsia', 'orange', 'dodgerblue', 
          'yellow', 'orchid', 'indigo', 'aqua','palegreen', 'silver', 'plum', 'fuchsia', 'coral',
          'gold', 'pink','slategray', 'forestgreen','peachpuff','honeydew','brown','olivedrab',
          'darkturquoise', 'tan', 'springgreen', 'mintcream','navajowhite','chocolate','lightblue','chartreuse',
          'lime','yellowgreen','khaki','gold','teal','tomato']
colors_items = [colors[ques_prefix.index(item.split('_')[0])] for item in ques_items]

for sample in ['Combined']:
    ols_res = ols_df[ols_df['sample'] == sample]

    fig, axs = plt.subplots(len(behs), 3, figsize=(15, 3*len(behs)), gridspec_kw={'width_ratios': [3, 3, 20]})
    fig.suptitle(f'{sample} sample', fontsize=18, y=1.01)
    if len(behs) == 1:
        ax1, ax2, ax3 = axs[0], axs[1], axs[2]
    else:
        ax1, ax2, ax3 = axs[i,0], axs[i,1], axs[i,2]

    for i, beh in enumerate(behs): 

        # questionnaire ~ behavior
        behav = ols_res[(ols_res['predictor_type'] == 'independent') & (ols_res['predicted'] == beh)]
        plot_weights(behav['beta'].values, colors, title='Scores independent', ax=ax1)
        
        # behavior ~ questionnaires
        behav = ols_res[(ols_res['predictor_type'] == 'covariate_scores') & (ols_res['predicted'] == beh)]
        plot_weights(behav['beta'].values, colors, title='Scores covariates', ax=ax2)
        
        # behavior ~ questionnaire items
        behav = ols_res[(ols_res['predictor_type'] == 'covariate_items') & (ols_res['predicted'] == beh)]
        plot_weights(behav['beta'].values, colors_items, title='Items covariates', ax=ax3)

    # overall legend
    handles, labels = plt.gca().get_legend_handles_labels()
    patches = [mpatches.Patch(facecolor=inst[0], edgecolor='black', label=inst[1]) for inst in zip(colors, ques)]  
    ax3.legend(title='', loc='upper right', handles=patches,  
                    title_fontsize=13, fontsize=8,
                    frameon=False, bbox_to_anchor=(1.15, 1), borderaxespad=0)
    plt.tight_layout()
    plt.show()

# Personality: agreeableness strongly correlates w/ distance 

In [ ]:
# not intolerance of uncertainty: ['ius_f1','ius_f2']

questions = [['mini_ipip_extraversion','mini_ipip_agreeableness','mini_ipip_conscientiousness',
              'mini_ipip_neuroticism','mini_ipip_intellect']]
for questions_ in questions:

    # plot
    fig, axs = plt.subplots(1, len(questions_), figsize=(3*len(questions_), 2.5))
    for q, ques in enumerate(questions_):
        ques_mask = data[ques] != 0.0
        sns.regplot(x='pov_2d_dist_mean_mean_z', y=ques, data=data[ques_mask],
                    scatter_kws={'alpha':0.3}, color='darkblue',
                    ax=axs[q])
    plt.tight_layout()
    plt.show()

    # run ols
    res_df, _ = run_ols(questions_, 'pov_2d_dist_mean_mean_z', data[ques_mask], covariates=demo_controls)
    display(res_df.round(3))

# Explore new things.... look at tendencies across characters for high social avoidance people

In [ ]:
# xy_cols = flatten_lists([[f'affil_mean_{role}', f'power_mean_{role}'] for role in character_roles[:5]])
# xys = data[xy_cols]

# # taking an average might be too simplistic...
# # it may be be better to try to capture some kind of curve or something... maybe can use other ratings (e.g., similarity )
# fig, ax = plt.subplots(1, 1, figsize=(4,4))

# for s in range(10,15):
#     xys_ = xys.iloc[s,:].values.reshape(5,2)
#     ax.scatter(*xys_.T, s=75, alpha=.5)
#     # maybe add some quadratic line or something...
#     plot_space_ax(ax)

# Brainstorming for modeling....

## Deliberation related to RT?
- Borderline, schiz related features: fewer symptoms, more rt
- self-perceivd status: less status, more rt

Can we trust online reaction times? Here, we know that we at least waited until resources were loadeed before prcoeeding... but of course other things can happen... so probably wont do too much here...

In [ ]:
mask_hi  = data['reaction_time_mean'] < 10
mask_low = data['reaction_time_mean'] > 2
data_    = data[mask_hi & mask_low]

scores = [c for c in data.columns if 'score' in c if 'iq' not in c]
for yvar in scores:

    ols_obj = run_ols(['reaction_time_mean'], yvar, data_, covariates=demo_controls)[1]
    pvalue  = ols_obj.pvalues[1]
    if pvalue < 0.01: 
        fig, axs = plt.subplots(1,2, figsize=(6,3))
        sns.histplot(data_['reaction_time_mean'], color='grey', bins=20, ax=axs[0])
        sns.regplot(x=data_['reaction_time_mean'], y=data_[yvar], scatter_kws={'alpha':0.2}, color='grey', ax=axs[1])
        plt.tight_layout()
        plt.show()
        
        display(ols_obj.summary())

## Subset based on social factor percentile and compare top 25% with bottm 25%

In [ ]:
mask   = np.isfinite(data[social_factor])
data_  = data[mask]
bottom = np.percentile(data_[social_factor], 20)
top    = np.percentile(data_[social_factor], 80)
data['social_factor_bin'] = np.where(data[social_factor] < bottom, 'low', np.where(data[social_factor] > top, 'high', 'middle'))

n_low = np.sum(data['social_factor_bin'] == 'low')
n_mid = np.sum(data['social_factor_bin'] == 'middle')
n_hi  = np.sum(data['social_factor_bin'] == 'high')

print(f'Low: {n_low} ({n_low/len(data)*100:.2f}%)')
print(f'Mid: {n_mid} ({n_mid/len(data)*100:.2f}%)')
print(f'Hi:  {n_hi} ({n_hi/len(data)*100:.2f}%)')

#--------------------------------------------
# plot
#--------------------------------------------

fig, axs = plt.subplots(1, 2, figsize=(7, 4))

ax = axs[0]
plot_histplot(data[social_factor], ax=ax, color='grey', bins=20)
ax.axvline(bottom, color='black', linestyle='--')
ax.axvline(top, color='black', linestyle='--')

ax = axs[1]
sns.barplot(x='social_factor_bin', y='pov_2d_dist_mean_mean', 
            data=data, ax=ax, palette='Greys', edgecolor='black', linewidth=1)
sns.stripplot(x='social_factor_bin', y='pov_2d_dist_mean_mean',
              data=data, ax=ax, palette='Greys', s=5, edgecolor='black', linewidth=1, alpha=.25)
ax.set_xlabel('Social factor', fontsize=label_fontsize)
# ax.set_xticklabels(['Low', 'Middle', 'High'])
ax.set_ylim(3.3, 8)
plt.show()

## Maybe small relationships to task memory, task engagement etc

slight positive trend w/ memory and w/ relatability 

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12, 3))

# memory for task
ax = axs[0]
sns.regplot(x='memory_mean', y='affil_mean_mean', data=data, ax=ax)
sns.regplot(x='memory_mean', y='power_mean_mean', data=data, ax=ax)
ax.set_ylabel('Dimension tendency')

# engagement with task
storyline_cols = ['storyline_engagement', 'storyline_difficulty', 'storyline_relatability']
mask = np.isfinite(data['storyline_difficulty'])
for c, col in enumerate(storyline_cols):
    ax = axs[1+c]
    sns.regplot(x=col, y='affil_mean_mean', data=data[mask], ax=ax)
    sns.regplot(x=col, y='power_mean_mean', data=data[mask], ax=ax)
    ax.set_ylabel('')

plt.tight_layout()

In [ ]:
# # trying to relate beta dissimilarities to semantics using an RSa
# ols_items['abs_beta'] = ols_items['beta'].abs()
# ols_items_ = ols_items.pivot(index=['item', 'text'], columns='dimension', values='abs_beta').reset_index()
# ols_items_.columns.name = None
# item_rdm = pairwise_distances(ols_items_[['affil_mean_mean', 'power_mean_mean']], metric='cosine') # captures the similarity in the relationship strength between items and dimensions
# plot_rdm(item_rdm, figsize=(5,5));

# vectors = get_sentencetransformer_embeddings(ols_items_['text'].values, model='roberta')
# semantic_rdm = pairwise_distances(vectors, metric='cosine')
# item_rdv = symm_mat_to_ut_vec(item_rdm)
# semantic_rdv = symm_mat_to_ut_vec(semantic_rdm)
# tau, p = kendalltau(item_rdv, semantic_rdv, alternative='greater')
# print(f'Kendall\'s tau: {tau:.3f}, p={p:.3f}')

# # IS-RSA
# ques_df = subset_df(data, fa_prefixes)
# ques_items = ques_df.iloc[:, 1:].values
# ques_items = np.nan_to_num(ques_items, nan=np.nanmean(ques_items, axis=0))
# ques_rdm = pairwise_distances(ques_items, metric='cosine')
# # plot_rdm(ques_rdm, figsize=(8, 8));

# ols_res, ols_individ_res = [], []
# for yvar in ['affil_mean_mean', 'power_mean_mean']:
#     ols_df = run_ols(X=top_items, y=yvar, data=df)[0]
#     ols_res.append(ols_df[ols_df['x'].isin(top_items)])
#     for item in top_items:
#         ols_df = run_ols(X=item, y=yvar, data=df)[0]
#         ols_individ_res.append(ols_df[ols_df['x'].isin([item])])
# top_ques = np.unique([q.split('_')[0] for q in top_items])
# print(top_ques)

# # turn ols_items into wide format
# ols_items_wide = ols_items.pivot(index='item', columns='dimension', values='beta')
# ols_items_wide = ols_items_wide.reset_index()
# ols_items_wide.columns.name = None

# scaler = MinMaxScaler(feature_range=(-1, 1))
# ols_items_wide['affil_mean_mean_scaled'] = scaler.fit_transform(ols_items_wide['affil_mean_mean'].values[:,np.newaxis])
# ols_items_wide['power_mean_mean_scaled'] = scaler.fit_transform(ols_items_wide['power_mean_mean'].values[:,np.newaxis])

# fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# sns.scatterplot(ax=ax, 
#                 x=ols_items_wide['affil_mean_mean_scaled'], 
#                 y=ols_items_wide['power_mean_mean_scaled'], s=75, alpha=1)
# plot_social_space(ax);

# Semantic embeddings
probably too naive of an approach rn... have to transform the questionnaire text appropriately - how do I account for the diff. question types etc?
- some items are reverse coded
- others had reverse rating instructions... e.g., how much you avoid this, etc...
- so the scores and text relationship might need to be flipped for these 

In [ ]:
def plot_cumulative_variance(X, standardize=True, plot=True, tangent=True, title=None):
    """
    Perform PCA and plot the cumulative variance explained as a function of the number of components.

    Parameters:
    - X: ndarray
        The input data matrix where each row is a sample and each column is a feature.
    - standardize: bool
        Whether to standardize the data before performing PCA.

    Returns:
    - cumulative_variance: ndarray
        Array containing the cumulative variance explained by each component.
    """
    
    # Standardize the data if needed
    if standardize:
        scaler = sklearn.preprocessing.StandardScaler()
        X = scaler.fit_transform(X)
    
    # Perform PCA & calculate the variance explained
    pca = PCA().fit(X)
    explained_variance  = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    
    # Plotting
    if plot:
        plt.figure(figsize=(8, 4))
        plt.bar(range(1, len(cumulative_variance) + 1), explained_variance, alpha=1, color='grey', label='Individual')
        plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', color='grey', label='Cumulative')
        plt.xlabel('Number PCs', fontsize=label_fontsize)
        # change x-ticks by + 1
        # plt.xticks(np.arange(1, len(cumulative_variance) + 1, 50))

        for color, cvar in {'orange': 0.5, 'orangered': 0.75, 'red': 0.9}.items():
            x = np.argmax(cumulative_variance >= cvar) + 1
            plt.scatter(x, cvar, marker='x', color=color, s=200, zorder=10)
            plt.text(x + 3, cvar, f'{int(cvar*100)}%={x} PCs', fontsize=15, color=color)

            if tangent:
                # add the tangent line at this point
                # find slope of tangent line by finding f'(x)
                # https://stackoverflow.com/questions/3985619/how-to-calculate-a-tangent-line-along-a-curve
                x1, y1 = x, cumulative_variance[x]
                x2, y2 = x+1, cumulative_variance[x+1]
                m = (y2 - y1) / (x2 - x1)
                x = np.linspace(0, 100, 100) # evaluate it at all these points
                y = m * (x - x1) + y1
                plt.plot(x, y, color=color, linestyle='-', linewidth=2, alpha=0.25)

        plt.ylim(0, 1)
        plt.ylabel('Explained Variance', fontsize=label_fontsize)
        title = title or 'Variance Explained'
        if standardize: title += ' (standardized)'
        plt.title(title, fontsize=title_fontsize)
        plt.legend()
        plt.show()

    return cumulative_variance
# plot_cumulative_variance(vectors);

In [ ]:
# t_value = 1
# n_items = np.sum(np.abs(ols_res['t']) > t_value)
# print(f'Number of items with |t| > {t_value}: {n_items}')

# # the beta weights for the top N items
# ols_rdm = pairwise_distances(ols_items_wide[['affil_mean_mean', 'power_mean_mean']].iloc[:n_items, :], metric='euclidean')
# ols_rdv = symm_mat_to_ut_vec(ols_rdm)

# # the embedding vectors
# vectors = get_sentencetransformer_embeddings(ols_items['text'].values[::2][:n_items], model='roberta')
# pca = PCA().fit_transform(vectors)
# embedding_rdm = pairwise_distances(pca[:, :10], metric='euclidean')
# embedding_rdv = symm_mat_to_ut_vec(embedding_rdm)

# # rsa
# tau, p = kendalltau(embedding_rdv, ols_rdv, alternative='greater')
# ols_perms = permute_symm_mat(ols_rdm, n_perms=1000)
# perm_taus = np.array([kendalltau(embedding_rdv, symm_mat_to_ut_vec(perm))[0] for perm in ols_perms])
# perm_p = calc_perm_pvalue(tau, perm_taus, alternative='greater')
# print(f'Tau = {tau:.3f}, p = {p:.3f}, perm p = {perm_p:.3f}')

# # plot
# fig, ax = plt.subplots(1, 1, figsize=(6, 4))
# plot_histplot(perm_taus, ax=ax);
# ax.axvline(tau, color='red', linestyle='--', linewidth=2)
# ax.set_xlabel('Kendall tau', fontsize=label_fontsize);

In [ ]:
# pca_df = pd.DataFrame(pca[:,:3], columns=['x', 'y', 'z'])
# fig = px.scatter_3d(pca_df, x='x', y='y', z='z', 
#                     # text=factor_top['text'].values,
#                     size_max=10, opacity=0.7)
# fig.update_layout(font = dict(family="Arial", size=10, color="black"))
# fig.show()

# Other correlations

In [ ]:
C = data_val[['pov_2d_dist_mean_mean', 'affil_mean_mean', 'power_mean_mean', 'memory_mean',
              'num_words_mean', 'compound_mean', 'positive_mean', 'negative_mean', 'neutral_mean',
               social_factor, mood_factor, compulsive_factor, 
              'dots_area', 'beh_dots_procrustes_disp', 'beh_dots_dist_memory_tau']].corr()
sns.heatmap(C, vmin=-1, vmax=1, cmap='coolwarm')
plt.show()

# WIP

In [ ]:
fig,axs = plt.subplots(2, 2, figsize=(4, 4), sharey=True)
fig.text(0.5, -0.05, 'Overlap', ha='center', fontdict={'fontsize':14})
fig.text(-0.05, 0.5, 'Memory', va='center', rotation='vertical', fontdict={'fontsize':14})
for i,r in enumerate([1,0,3,2]):
    ax = axs[i//2, i%2]
    sns.regplot(ax=ax, x=f'Q{r+1}_overlap', y='memory_mean', data=data, 
                line_kws={'color':'black', 'linewidth':2},
                color='grey', scatter_kws={'s':20, 'edgecolor':'black', 'alpha':.25})
    ax.set_title(f'Q{r+1}', fontdict={'fontsize':20})
    ax.set_xticks([0, .5, 1])
    ax.set_yticks([0, .5, 1])
    ax.set_xlabel('')
    ax.set_ylabel('')
plt.tight_layout()


### Relationship preferences

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6,3))
sns.histplot(ax=axs[0], x=data['prospective_choice_rt_avg'], bins=20)
sns.regplot(ax=axs[1], x='prospective_choice_rt_avg', y='prospective_choice_pref_avg', data=data,
            scatter_kws={'alpha':0.5, 's':30})
plt.tight_layout()

### People want the powerful person in their network most

In [ ]:
data_ = data[[f'prospective_{role}_mean' for role in character_roles]]

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.barplot(data_, alpha=0.5, palette='colorblind', edgecolor='black', linewidth=1, ci=None, ax=ax)
sns.stripplot(data_, palette='colorblind', edgecolor='black', linewidth=0.5, ax=ax)
ax.set_xticklabels(character_roles, rotation=80, horizontalalignment='right')
plt.show()

### Intrusive factor has a correlation w/ average preference (not strong though)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(9, 3))
for i, factor in enumerate(factor_labels): 
    plot_regplot(data['prospective_choice_pref_avg'], data[factor], ax=axs[i])
plt.tight_layout()

# Random

In [ ]:
# use MLPRegressor on all combinations of Xs to predict factor_social_min and choose best model by accuracy
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from itertools import combinations

Xs = ['likability_selfOther_avg', 'impact_selfOther_diff', 'pov_2d_dist_mean', 'affil_mean_mean', 'power_mean_mean']   
# get all possible combinations of Xs
Xs_combos = []
for i in range(1, len(Xs)+1):
    Xs_combos += list(combinations(Xs, i))

# store performane
performance = {}

# loop over Xs_combos and train MLPRegressor on each
for X in Xs_combos:

    # mask with np.isfinite
    mask = np.isfinite(data[list(X) + [social_factor]]).all(1)
    data_ = data[mask]

    # split data into train and test
    X_train, X_test, y_train, y_test = train_test_split(data_[list(X)], 
                                                        data_[social_factor], 
                                                        test_size=0.2, random_state=42)

    # MLPRegressor
    mlp = MLPRegressor(hidden_layer_sizes=(10,10), activation='relu', solver='adam', max_iter=100, random_state=42)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    
    # diff metrics
    mse = np.mean((y_test - y_pred)**2)
    r2 = r2_score(y_test, y_pred)
    r2_adj = adj_r2(r2, X_test.shape[0], X_test.shape[1])

    # join each element of X into a string
    X = ','.join(X)

    # store performance
    performance[X] = {'mse': mse, 'r2': r2, 'r2_adj': r2_adj}

In [ ]:
# sort the keys by r2_adj
performance = {k: v for k, v in sorted(performance.items(), key=lambda item: item[1]['r2_adj'], reverse=True)}
# get first key
best_model = list(performance.keys())[0]
print(f'Best model: {best_model}')
print(f'Performance: {performance[best_model]}')

# use best model to predict factor_social_min with linear regression
ols_obj = run_ols(best_model.split(','), social_factor, data, covariates=demo_controls)[1]
print(ols_obj.summary())

# -----------------------------------------------------------------------------------
# EXPERIMENTAL
# -----------------------------------------------------------------------------------

### CCA: good correlations - do more here?

In [ ]:
# from dim_reduction import CrossDecompose
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_decomposition import CCA, PLSRegression, PLSCanonical
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedShuffleSplit, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def clean_data(data, yvars, xvars, var_thresh=0.0):
    # X = data[xvars].dropna() # drop rows with nans
    # use np.isfinite to drop rows with nan
    
    fin_mask = np.isfinite(data[xvars + yvars]).all(1)
    conds = data[fin_mask]['sample'].values
    Y = data[fin_mask][yvars].reset_index(drop=True)

    # clean up X 
    X = data[fin_mask][xvars].reset_index(drop=True)
    vt = VarianceThreshold(threshold = var_thresh) #Removing both constant and quasi-constant
    vt.fit(X)
    X = X.loc[:, vt.get_support()]

    assert not np.any(np.isnan(X)), 'X has nans'
    assert not np.any(np.isnan(Y)), 'Y has nans'
    print(f'X shape: {X.shape}, Y shape: {Y.shape}')
    return X, Y, conds


In [ ]:
x_cols = [social_factor, 'demo_ladder_rate', 'sni_number_ppl', 'sni_network_diversity','likability_relationship_mean', 'impact_relationship_mean', 
          'self_likability', 'self_impact']
y_cols = ['pov_2d_dist_mean_mean', 'affil_mean_mean', 'power_mean_mean', 'pov_2d_angle_mean_mean']
yvars_ = [['affil_mean_mean', 'power_mean_mean'], ['pov_2d_dist_mean_mean']] #, ['pov_dist_mean', 'pov_2d_angle_mean_mean']]

for yvars in yvars_:

    # remove yvars frm xvars
    X, Y, conds = clean_data(data, yvars, x_cols, var_thresh=0.0)

    # split X and Y into train/test balancing on condition
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=35)
    splits = list(sss.split(np.arange(len(conds)), conds))
    train_ix, test_ix = splits[0][0], splits[0][1]

    X_train, Y_train = X.iloc[train_ix, :], Y.iloc[train_ix, :]
    X_test, Y_test   = X.iloc[test_ix, :], Y.iloc[test_ix, :]
    conds_test, conds_train = conds[test_ix], conds[train_ix]

    assert len(X_train) == len(Y_train), 'X_train and Y_train have different lengths'
    assert len(X_test) == len(Y_test), 'X_test and Y_test have different lengths'

    #--------------------------------------------------------------------------------------
    # run pls: CV to select n_comps, then validate on test_set
    #--------------------------------------------------------------------------------------

    # zscore X and Y for train and test separately
    X_train, Y_train = scipy.stats.zscore(X_train), scipy.stats.zscore(Y_train) 
    X_test, Y_test   = scipy.stats.zscore(X_test), scipy.stats.zscore(Y_test) 

    assert not np.any(np.isnan(X_train)), 'X_train has nans'
    assert not np.any(np.isnan(Y_train)), 'Y_train has nans'
    assert not np.any(np.isnan(X_test)), 'X_test has nans'
    assert not np.any(np.isnan(Y_test)), 'Y_test has nans'

    # select n_comps w/ 5 fold CV on training data
    loadings = {}
    pls = CrossDecompose(n_comps=5, scale=False, mode='pls')
    pls.optimize_fit(X_train, Y_train, max_comps=None, folds=5, metric='MSE', plot=False, colors=None)
    # pls.fit(X_train, Y_train)
    loadings['optimization'] = pls.X_loadings
    top_features = pls.get_top_features(n_features=30) # get top features for diff components

    _ = pls.predict(X_test, Y_test, plot=True)
    pls.plot_X_loadings()
    

NameError: name 'clean_data' is not defined

In [ ]:
from statsmodels.multivariate.cancorr import CanCorr
Y = data[['likability_relationship_mean', 'impact_relationship_mean']] # y is the relationship likability and impact

Xs = [['self_likability', 'self_impact'], 
      ['affil_mean_mean', 'power_mean_mean']]
for X_ in Xs:
    X = data[X_] # X is self likability and self impac
    mask = (np.sum(np.isfinite(X),1) == X.shape[1]) & (np.sum(np.isfinite(Y),1) == Y.shape[1])
    cca = CanCorr(X[mask], Y[mask])
    print(cca.corr_test().summary())

## Data consistent with a mediation of this sort: self-place -> social approach/avoid -> behavioral distancing/SNI etc...

In [ ]:
def run_mediation(x, m, y, df, covs):

    #--------------------------------------------------
    # preprpocess the data

    data = df.copy()
    
    # mask out non-responses
    all_vars = [x,y,m] + covs
    mask = np.sum(np.isfinite(data[[x,y,m] + covs]), 1) == len(all_vars)
    data = data[mask]

    print(f'testing: {x} -> {m} -> {y}')
    print(f'n={data.shape[0]}')

    # dummy code, zscore
    for v in [x, y, m] + covs:
        if type(v) == str: data[v] = data[v].astype('category').cat.codes        
        data[v] = zscore(data[v])

    #--------------------------------------------------
    # run mediation analysis
    # - y ~ x   
    reg = run_ols([x], y, data, covariates=covs)[1]
    x_ = [n for n in reg.pvalues.index if x in n][0] # jic its categorical
    [p1, b1] = reg.pvalues[x_], reg.params[x_]

    # - m ~ x
    reg = run_ols([x], m, data, covariates=covs)[1]
    [p2, b2] = reg.pvalues[x_], reg.params[x_]

    # - y ~ m
    reg = run_ols([m], y, data, covariates=covs)[1]
    [p3, b3] = reg.pvalues[m], reg.params[m]

    # - y ~ x + m
    reg = run_ols([x, m], y, data, covariates=covs)[1]
    [p4, b4] = reg.pvalues[x_], reg.params[x_]
    
    # stack the ps and betas
    res = np.array([[p1, p2, p3, p4], 
                    [b1, b2, b3, b4]]).T

    # make a dataframe
    res = pd.DataFrame(res, columns=['p', 'beta(z)'], index=['y~x', 'm~x', 'y~m', 'y~x+m'])
    res = res.applymap(lambda x: '{:.7f}'.format(x))
    return res

In [ ]:
# # can we do a mediation analysis with only measured features? e.g., no causal manipulaion...
# # self location -> social avoidance -> behavioral distancing, network features, etc...

# models = [['self_impact', 'demo_ladder_rate', 'factor_social'],
#           ['demo_ladder_rate', 'self_impact', 'factor_social'],
#           ['self_likability', 'demo_ladder_rate', 'factor_social'],
#           ['demo_ladder_rate', 'self_likability', 'factor_social'],
#           ['self_impact', 'demo_ladder_rate', 'pov_2d_dist_mean'],
#           ['self_likability', 'demo_ladder_rate', 'pov_2d_dist_mean'],
#           ['self_impact', 'factor_social', 'pov_2d_dist_mean'],
#           ['factor_social', 'self_impact', 'pov_2d_dist_mean'],
#           ['self_likability', 'factor_social', 'pov_2d_dist_mean'],
#           ['factor_social', 'self_likability', 'pov_2d_dist_mean']]

# ys = ['sni_network_diversity', 'sni_number_ppl']

# for mod in models:
#     display(run_mediation(mod[0], mod[1], mod[2], data, demo_controls))

In [ ]:
from statsmodels.stats.mediation import Mediation

# ACME: average causal mediation effect - the indirect effect of X on Y through M
# ADE: average direct effect - direct effect of X on Y
# total effect: driect + indirect of X on Y
# prop mediated: proportion of effect of X on Y mediated by M

x, m, y  = 'self_likability', social_factor, 'pov_2d_dist_mean_mean_z'
all_vars = [x, y, m] + demo_controls
mask     = np.sum(np.isfinite(data[all_vars]), 1) == len(all_vars)

# family = sm.families.Binomial(link=links.probit())
demo_str = (' + ').join(demo_controls)
outcome_model  = sm.GLM.from_formula(f"{y} ~ {x} + {m} + {demo_str}", data[mask])
mediator_model = sm.OLS.from_formula(f"{m} ~ {x} + {demo_str}",  data[mask])
mediation = Mediation(outcome_model, mediator_model, x, m).fit()
mediation.summary()